#### **Merging and Joining**

1. **Inner Join on Chinook Database**
   - Load the `chinook.db` database.
   - Perform an inner join between the `customers` and `invoices` tables on the `CustomerId` column.
   - Find the total number of invoices for each customer.

In [3]:
import pandas as pd
import sqlite3 as sql

In [ ]:
#We can upload customers and invoices tables separately:
with sql.connect('data/chinook.db') as connection:
    df_customers = pd.read_sql(
        "SELECT * FROM customers",
        con = connection)
    df_invoices = pd.read_sql(
        "SELECT * FROM invoices",
        con = connection)
#We will join two tables using merge():
df_joined = pd.merge(df_customers,df_invoices,on='CustomerId',how='inner')
#Now, we will group each customer by their ID and find total number of invoices:
group_customers = df_joined.groupby('CustomerId')['Total']
group_customers.sum()

2. **Outer Join on Movie Data**
   - Load the `movie.csv` file.
   - Create two smaller DataFrames:
     - One with only `director_name` and `color`.
     - Another with `director_name` and `num_critic_for_reviews`.
   - Perform a left join and then a full outer join on `director_name`.
   - Count how many rows are in the resulting DataFrames for each join type.

In [33]:
#Loading:
df_movie = pd.read_csv('data/movie.csv')
#Two smaller DataFrame:
df_director_color = df_movie[['director_name','color']]
df_director_critic = df_movie[['director_name','num_critic_for_reviews']]
#Joining two DataFrame using merge:
left_joined_df = pd.merge(df_director_color,df_director_critic,on='director_name',how='left')
outer_joined_df = pd.merge(df_director_color,df_director_critic,on='director_name',how='outer')
#Counting number of rows of each new joined DataFrame:
print(f'The number of rows of the left joined DataFrame:{left_joined_df.shape[0]}')
print(f'The number of rows of the outer joined DataFrame:{outer_joined_df.shape[0]}')

The number of rows of the left joined DataFrame:30300
The number of rows of the outer joined DataFrame:30300


#### **Grouping and Aggregating**

1. **Grouped Aggregations on Titanic**
   - Group passengers by `Pclass` and calculate the following:
     - Average age.
     - Total fare.
     - Count of passengers.
   - Save the results to a new DataFrame.


In [99]:
#Loading data:
df_titanic = pd.read_excel('data/titanic.xlsx')
#Grouping passengers by Pclass:
result = df_titanic.groupby('Pclass').agg({
    'Age' : 'mean',
    'Fare' : 'sum',
    'Name' : 'size'
})
#We need to make Pclass a new colunm and change result accordingly:
result.reset_index(drop=False,inplace=True)
values = (list(result.iloc[0])),list(result.iloc[1]),list(result.iloc[2])
#Now, we can upload data into a new databas file:
with sql.connect('info_titanic.db') as connection:
    cursor = connection.cursor()
    cursor.execute("DROP TABLE IF EXISTS Pclass")
    cursor.execute("""CREATE TABLE Pclass(
                   Pclass INT,
                   AverageAge FLOAT,
                   TotalFare FLOAT,
                   NumberOfPassengers INT);""")
    cursor.executemany("INSERT INTO Pclass VALUES(?,?,?,?)",values)
    connection.commit()

2. **Multi-level Grouping on Movie Data**
   - Group the movies by `color` and `director_name`.
   - Find:
     - Total `num_critic_for_reviews` for each group.
     - Average `duration` for each group.

In [112]:
# Group by both color and director_name:
group = df_movie.groupby(['color', 'director_name'])

# Total number of critic reviews per (color, director) group
print('Total number of critics for reviews based on color and director_name:')
print(group['num_critic_for_reviews'].sum())

print('-' * 50)

# Average duration per (color, director) group
print('Average duration of movies based on color and director_name:')
print(group['duration'].mean())

Total number of critics for reviews based on color and director_name:
color            director_name     
Black and White  Akira Kurosawa        153.0
                 Aleksey German        121.0
                 Alex Garland          489.0
                 Alexander Payne       433.0
                 Alfred Hitchcock      434.0
                                       ...  
Color            Zoran Lisinac          17.0
                 Álex de la Iglesia     71.0
                 Émile Gaudreault       67.0
                 Éric Tessier            9.0
                 Étienne Faure           9.0
Name: num_critic_for_reviews, Length: 2490, dtype: float64
--------------------------------------------------
Average duration of movies based on color and director_name:
color            director_name     
Black and White  Akira Kurosawa        202.0
                 Aleksey German        177.0
                 Alex Garland          108.0
                 Alexander Payne       115.0
            

3. **Nested Grouping on Flights**
   - Group flights by `Year` and `Month` and calculate:
     - Total number of flights.
     - Average arrival delay (`ArrDelay`).
     - Maximum departure delay (`DepDelay`).


In [38]:
#Loading flights file:
df_flights = pd.read_parquet('data/flights')

In [ ]:
#Group by Year and month:
group_year_month = df_flights.groupby(['Year','Month'])

# Combine all metrics into one DataFrame
summary_df = group_year_month.agg({
    'Flights': 'sum',
    'ArrDelay': 'mean',
    'DepDelay': 'max'
})

print(summary_df)

#### **Applying Functions**

1. **Apply a Custom Function on Titanic**
   - Write a function to classify passengers as `Child` (age < 18) or `Adult`.
   - Use `apply` to create a new column, `Age_Group`, with these values.

In [7]:
#Loading data:
df_titanic = pd.read_excel('data/titanic.xlsx')

#This function will classify passengers based on their age:
def classify(x):
    if x < 18:
        return 'Child'
    else:
        return 'Adult'

#Applying the function to Age Series and we can append the result to a new column:
df_titanic["Age_Group"] =  df_titanic['Age'].apply(classify)

2. **Normalize Employee Salaries**
   - Load the `employee.csv` file.
   - Normalize the salaries within each department.

In [ ]:
#Loading the file:
df_employee = pd.read_csv('data/employee.csv')

#first, we group the DataFrame by each department and then transform Min-Max normalize function:
df_employee['Salary_Normalized'] = df_employee.groupby('DEPARTMENT')['BASE_SALARY'].transform(lambda x: (x - x.min())/(x.max()-x.min()))

df_employee.head()

,UNIQUE_ID,POSITION_TITLE,DEPARTMENT,BASE_SALARY,RACE,EMPLOYMENT_TYPE,GENDER,EMPLOYMENT_STATUS,HIRE_DATE,JOB_DATE,Salary_Normalized
0,0,ASSISTANT DIRECTOR (EX LVL),Municipal Courts Department,121862.0,Hispanic/Latino,Full Time,Female,Active,2006-06-12,2012-10-13,1.000000
1,1,LIBRARY ASSISTANT,Library,26125.0,Hispanic/Latino,Full Time,Female,Active,2000-07-19,2010-09-18,0.000000
2,2,POLICE OFFICER,Houston Police Department-HPD,45279.0,White,Full Time,Male,Active,2015-02-03,2015-02-03,0.116351
3,3,ENGINEER/OPERATOR,Houston Fire Department (HFD),63166.0,White,Full Time,Male,Active,1982-02-08,1991-05-25,0.192491
4,4,ELECTRICIAN,General Services Department,56347.0,White,Full Time,Male,Active,1989-06-19,1994-10-22,0.479189


3. **Custom Function on Movies**
   - Write a function that returns `Short`, `Medium`, or `Long` based on the duration of a movie:
     - `Short`: Less than 60 minutes.
     - `Medium`: Between 60 and 120 minutes.
     - `Long`: More than 120 minutes.
   - Apply this function to classify movies in the `movie.csv` dataset.


In [22]:
#Loading:
df_movie = pd.read_csv('data/movie.csv')

In [27]:
#This function will classify movies based on their duration:
def duration(x):
    if pd.isna(x):
        return 'Not given'
    elif x < 60:
        return 'Short'
    elif 60< x < 120:
        return 'Medium'
    else:
        return 'Long'

#Applying this function to the DataFrame:
df_movie['Duration_Classified'] = df_movie['duration'].apply(duration)
df_movie.head()

,color,director_name,num_critic_for_reviews,duration,director_facebook_likes,actor_3_facebook_likes,actor_2_name,actor_1_facebook_likes,gross,genres,...,language,country,content_rating,budget,title_year,actor_2_facebook_likes,imdb_score,aspect_ratio,movie_facebook_likes,Duration_Classified
0,Color,James Cameron,723.0,178.0,0.0,855.0,Joel David Moore,1000.0,760505847.0,Action|Adventure|Fantasy|Sci-Fi,...,English,USA,PG-13,237000000.0,2009.0,936.0,7.9,1.78,33000,Long
1,Color,Gore Verbinski,302.0,169.0,563.0,1000.0,Orlando Bloom,40000.0,309404152.0,Action|Adventure|Fantasy,...,English,USA,PG-13,300000000.0,2007.0,5000.0,7.1,2.35,0,Long
2,Color,Sam Mendes,602.0,148.0,0.0,161.0,Rory Kinnear,11000.0,200074175.0,Action|Adventure|Thriller,...,English,UK,PG-13,245000000.0,2015.0,393.0,6.8,2.35,85000,Long
3,Color,Christopher Nolan,813.0,164.0,22000.0,23000.0,Christian Bale,27000.0,448130642.0,Action|Thriller,...,English,USA,PG-13,250000000.0,2012.0,23000.0,8.5,2.35,164000,Long
4,NaN,Doug Walker,NaN,NaN,131.0,NaN,Rob Walker,131.0,NaN,Documentary,...,NaN,NaN,NaN,NaN,NaN,12.0,7.1,NaN,0,Not given


#### **Using `pipe`**
1. **Pipeline on Titanic**
   - Create a pipeline to:
     - Filter passengers who survived (`Survived == 1`).
     - Fill missing `Age` values with the mean.
     - Create a new column, `Fare_Per_Age`, by dividing `Fare` by `Age`.


In [36]:
df_titanic = (
df_titanic
    .loc[lambda x: x['Survived'] == 1]
    .assign(Age=lambda x: x['Age'].fillna(x['Age'].mean()))
    .pipe(lambda x: x.assign(Fare_Per_Age = x['Fare'] / x["Age"]))
    .reset_index(drop=True)
)
df_titanic.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,Age_Group,Fare_Per_Age
0,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C,Adult,1.875876
1,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S,Adult,0.304808
2,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S,Adult,1.517143
3,9,1,3,"Johnson, Mrs. Oscar W (Elisabeth Vilhelmina Berg)",female,27.0,0,2,347742,11.1333,NaN,S,Adult,0.412344
4,10,1,2,"Nasser, Mrs. Nicholas (Adele Achem)",female,14.0,1,0,237736,30.0708,NaN,C,Child,2.147914


2. **Pipeline on Flights**
   - Create a pipeline to:
     - Filter flights with a departure delay greater than 30 minutes.
     - Add a column `Delay_Per_Hour` by dividing the delay by the scheduled flight duration.

In [46]:
df_filtered_flights = (
df_flights
        .assign(
        DepDelayMinutes=lambda x: pd.to_numeric(x['DepDelayMinutes'], errors='coerce'),
        ActualElapsedTime=lambda x: pd.to_numeric(x['ActualElapsedTime'], errors='coerce'))
        .loc[lambda x: x['DepDelayMinutes'] > 30]
        .pipe(lambda x: x.assign(Delay_Per_Hour = x['DepDelayMinutes'] / x['ActualElapsedTime']))
        .reset_index(drop = True)
)